##### Login Crab

In [ ]:
import requests
from IPython.core.display import display, HTML

params = {'username': 'aaa','password':'password'}
r = requests.post("http://www.pythonscraping.com/pages/cookies/welcome.php", params)
print("cookiese is set to: ")
print(r.cookies.get_dict())
print('Going to profile page...')
r = requests.get("http://www.pythonscraping.com/pages/cookies/welcome.php")

display(HTML(r.text))

In [ ]:
import requests
from IPython.core.display import display, HTML

session = requests.Session()
params = {'username': 'username','password':'password'}
s = session.post("http://www.pythonscraping.com/pages/cookies/welcome.php", params)
print("cookiese is set to: ")
print(s.cookies.get_dict())
print('Going to profile page...')

s = session.get("http://www.pythonscraping.com/pages/cookies/welcome.php")

display(HTML(s.text))

In [ ]:
import requests
from requests.auth import AuthBase
from requests.auth import HTTPBasicAuth
auth = HTTPBasicAuth('ryan', 'password')
r = requests.post(url="http://www.pythonscraping.com/pages/auth/login.php",auth=auth)
print(r.text)

##### Catch Data and Create Database

In [ ]:
#此程式有使用google瀏覽器套件
#不能在notebook上執行
#目的為抓取網路店上ZenBook系列筆電的資料
#結果的csv檔，是我用自己的電腦跑完再上傳到notebook上的
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup as bs
import requests
import csv
import os
import pymysql

username = 'root'
password = 'DB password'
databaseName = 'DB name'


chrome_option = Options()
chrome_option.add_argument('--headless')#不希望出現視窗
driver = webdriver.Chrome('chromedriver',chrome_options = chrome_option)#透過瀏覽器開啟

driver.get('https://store.asus.com/tw/category/A15061')#Asus的官方網路店
time.sleep(3)#等待網頁載入完成

content = bs(driver.page_source,'html.parser')#透過BeautifulSoup開啟
linkList = content.select('a.photo')#抓取各種筆電的連結
urls = []
for i in linkList:
    urls.append(i.get('href'))#把連結放進urls
#print(urls)
driver.close()#關閉瀏覽器


wantlist = ['型號','價格','CPU',"GPU","RAM","資料儲存應用",'保固']#這是我想找的資料，也是csv的第一列

db = pymysql.connect('localhost',username,password,databaseName)
cursor = db.cursor()

tmp = []#用於暫存從網頁擷取下來的資料
width = [0]*7#用於控制excel欄位寬度
for url in urls: #透過urls，瀏覽每一台筆電的詳情
    web = requests.get("https:" + url)#請求網頁
    content = bs(web.text,"html.parser")#用湯開啟
    name = content.find('h1',id='pro_title').text#找到產品名稱
    tmp.append(name)#放入暫存
    #print(name)
    price = content.find("span",{'class':"price"}).text#找到價格
    tmp.append(price)
    #print(price)
    item = content.select(".css-spec-item")#找到詳細資料標頭
    itemData = content.select(".css-spec-data")#找到詳細資料
    for i in range(len(item)):#將兩個list文字化(.text)
        item[i] = item[i].text
        itemData[i] = itemData[i].text.replace('\n',"")
    check = 0#用於確定資料完整
    for i in range(len(item)):
        if item[i] in wantlist:#篩選想要的資料
            #print(item[i] + ":" + itemData[i])
            tmp.append(itemData[i])
            check += 1
    if check == 5:#如果資料完整
        tmp[1] = int(tmp[1].replace(',',""))
        print(tmp)
        cursor.execute('insert into laptop value(%s,%s,%s,%s,%s,%s,%s,null)',[tmp[i] for i in range(7)])
    tmp.clear()#清空暫存
db.commit()

# crawler.py
# 目前顯示的是「crawler.py」。

##### Link Database and Find by CPU

In [ ]:
import pymysql
import pandas as pd
username = 'root'
password = 'hongwenwang'
databaseName = "comp"
# connectDB()
db = pymysql.connect('localhost',username,password,databaseName)
cursor = db.cursor()
print("longin suceed")

In [ ]:
def get_keyword_activity(keyword=None):
    msg=""
    if "i7" in keyword:
        cursor.execute("SELECT * FROM laptop")
        result = cursor.fetchall()
        for row in result:
            for i in row:
                msg += str(i)
    return msg

In [ ]:
import json
from flask import Flask, escape, request, make_response, jsonify

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

@app.route('/', methods=['POST'])
def webhook():
    json = request.get_json(silent=True,force=True)
    print(json)
    
    CPU = None
#     產品系列
    if json['queryResult']['parameters']['Brand']!= '' :
        Brand = json['queryResult']['parameters']['Brand']
    
#     GPU
    if json['queryResult']['parameters']['GPU']!='':
        GPU = json['queryResult']['parameters']['GPU']
    
#     價格
    if json['queryResult']['parameters']['number']!='':
        number = json['queryResult']['parameters']['number']
    
#     CPU
    if json['queryResult']['parameters']['CPU']!='':
        CPU = json['queryResult']['parameters']['CPU']
        
#     作業系統
    if json['queryResult']['parameters']['OS']!='':
        OS = json['queryResult']['parameters']['OS']
    
#     硬碟
    if json['queryResult']['parameters']['Disk']!='':
        disk = json['queryResult']['parameters']['Disk']
        
#     保固
    if json['queryResult']['parameters']['any']!='':
        warr = json['queryResult']['parameters']['any']

#     特價表
#     if json['queryResult']['parameters']['discount']!='':
#         discount = json['queryResult']['parameters']['discount']
        
#     關注清單
#     if json['queryResult']['parameters']['attention']!='':
#         attention = json['queryResult']['parameters']['attention']
        
#     執行動作
    if json['queryResult']['parameters']['action']!='':
        action = json['queryResult']['parameters']['action']

#     比較
    if json['queryResult']['parameters']['compare']!='':
        compare = json['queryResult']['parameters']['compare']
    
#     日期
#     if json['queryResult']['parameters']['date']!='':
#         date = parser.parse(json['queryResult']['parameters']['date'])
#         start = date.date()
#     elif json['queryResult']['parameters']['date-period']!='':
#         start = parser.parse(json['queryResult']['parameters']['date-period']['startDate']).date()
#         end = parser.parse(json['queryResult']['parameters']['date-period']['endDate']).date()
    
    res_message = {"fulfillmentText":get_keyword_activity(CPU)}
    
    print(res_message)
    return make_response(jsonify(res_message))

In [ ]:
if __name__ == "__main__":
    app.run(port=5000)

##### BeautifulSoup find function

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

html = requests.get("https://www.acer.com/ac/zh/TW/content/models/laptops")# Acer 筆電官網
soup = bs(html.text,"lxml")
div = soup.find("div",{"class","product-list"}) 

In [ ]:
img = div.find_all("img",{"class","centerImage zh-heightImg"})#圖片
#圖片處理
imgLink=[]
for row in img:
    imgLink.append(row.get("src").strip())
# print(imgLink)
len(imgLink)

In [ ]:
types = div.find_all("h2")#型號
#型號處理
ID=[]
for row in types:
    ID.append(row.text.strip())
len(ID)

In [ ]:
content = div.find_all("a",{"class", "zh-heightContImg"})
conLink = []
for row in content:
    conLink.append( "https://www.acer.com/ac/" + row.get("href").strip())
len(conLink)

In [ ]:
page = requests.get(conLink[0])
# time.sleep(3)
soup = bs(html.text,"lxml")
service = soup.find("div",{"id","printableArea"}) 
# print(soup.text)
soup

##### More Infomation Catch

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

html = requests.get("https://www.acer.com/ac/zh/TW/content/models/laptops")# Acer 筆電官網
soup = bs(html.text,"lxml")
div = soup.find("div",{"class","product-list"}) #將網頁中 div 找出

In [ ]:
img = div.find_all("img",{"class","centerImage zh-heightImg"})#圖片
#圖片處理
imgLink=[]
for row in img:
    imgLink.append(row.get("src").strip())
print(imgLink)


In [ ]:
types = div.find_all("h2")#型號
#型號處理
Type=[]
for row in types:
    Type.append(row.text.strip())
print(Type[:5])

In [ ]:


ul = div.find_all("ul",{"class","text-m text-tertiary-dark align-center ctnTechSpecList"})

OS = []
CPU = []
View = []
RAM = []
Disk = []

for row in ul:
#     print(row)
    fields = row.find_all("span")#找到所有li
    os = 1
    cpu = 1
    view = 1
    ram = 1
    disk = 1
    for i in fields: # 有些筆電格是不一樣 li 不一致 所以找關鍵字
        t = i.text.strip()
        if "Windows" in t or "Chrome" in t: 
            OS.append(t)  # OS
            os = 0
        if "核心" in t or "處理器" in t:
            CPU.append(t)  # CPU
            cpu =0
        if "Graphics" in t or "NVIDIA" in t:
            View.append(t)  # viewCard
            view = 0
        if "DDR" in t :
            RAM.append(t)  # RAM
            ram =0
        if "SSD" in t or "HDD" in t or "快閃記憶體" in t:
            Disk.append(t)  # Disk
            disk = 0
    if os == 1:#那台電腦配備未列出時
        OS.append("")
    if cpu == 1:
        CPU.append("")
    if view == 1:
        View.append("")
    if ram == 1:
        RAM.append("")
    if disk == 1:
        Disk.append("")

In [ ]:

#欄位標題
row_title = [] 
row_title.append("型號")
row_title.append("圖片連結")
row_title.append("作業系統")
# row_title.append("價格")
row_title.append("顯示卡")
row_title.append("CPU")
row_title.append("RAM")
row_title.append("DISK")
#row_title.append("SSD")
#row_title.append("HDD")
# row_title.append("保固期")

In [ ]:
notebook = pd.DataFrame({
    row_title[0]:Type,
    row_title[1]:imgLink,
    row_title[2]:OS,
    row_title[3]:View,
    row_title[4]:CPU,
    row_title[5]:RAM,
    row_title[6]:Disk,
})
# print(csv.head())
# notebook.to_csv("notebook.csv",index = False,sep='\t',encoding="utf-8") # 用excel會有亂碼
notebook.to_csv("notebook.csv",index = False,sep=',',encoding="utf_8_sig") #sep為分隔符號 用excel不會有亂碼

##### Completely Crab Infomation and Process data

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [ ]:
def get_keyword_activity(keyword = None):
    
    html = requests.get("http://www.oiac.fcu.edu.tw/zh-tw/alliances/2")
    soup = bs(html.text,"lxml")

    table = soup.find("div", {"id" : "grids"}) #將網頁中 div 找出

    #欄位標題
    row_title = [] 
    row_title.append("名稱")
    row_title.append("連結")
   
    #抓取欄位值

    fields = []
    links = []
    name = []
    
    for row in table.find_all("strong"):
        fields = row.find("a")            #廠商找到連結和名稱
        name.append(fields.text.strip())  #加到list
        if "http" not in fields.get("href"):#得到連結不完整
            links.append("NULL")
        else:
            links.append(fields.get("href").replace("#http","http"))#得到連結，若連結是 #http 改為 http

    company_df = pd.DataFrame({row_title[0]:name, row_title[1]:links})#建立 Dataframe

    
    msg = ""

    if keyword != None and keyword != "total":
        for i,name in enumerate(company_df["名稱"]):
            if keyword in name:   #判斷 keyword 是否包含在 name 中
                if msg == "":
                    msg = "有找到公司!!!\n"
                msg +=  company_df["名稱"][i] + '\n' + company_df["連結"][i] + ' \n'
    elif keyword == "total":   # keyword 在 dialogflow 中是 total的 entity會傳全部
        for indax,activity in company_df.iterrows():
            msg += company_df["名稱"][indax] + ' ' + company_df["連結"][indax] + '\n' 
    else:             # 沒有 keyword 時
        msg = "enter the company"
        return msg
    msg.strip() 
    print(msg)
    if msg == "":
        msg = "沒有這間公司"
        

    return msg[:300] #保證不超過ngrok的大小限制

In [ ]:
import json
from flask import Flask, escape, request, make_response, jsonify

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

@app.route('/', methods=['POST'])
def webhook():
    
    json = request.get_json(silent=True,force=True)
    print(json)
    keyword = None
    if json['queryResult']['parameters']['any']!='':  #得到在 dialogflow 為 any 的 entity
        keyword = json['queryResult']['parameters']['any']
    elif json['queryResult']['parameters']['total']!='':#得到在 dialogflow 為 total 的 entity
        keyword = "total"  #將 keyword 設為 total
    
    res_message = {"fulfillmentText":get_keyword_activity(keyword=keyword)} #得到要回傳訊息
    
    return make_response(jsonify(res_message))#回傳

In [ ]:
# need using dialogflow
if __name__ == "__main__":
    app.run(port=5000)

##### Word Analysis

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [ ]:
r = requests.get("https://www.cw.com.tw/article/article.action?id=5071446")
soup = bs(r.text,"lxml")
div = soup.find("div",{'class':None,"id":None})

st = div.text.strip()# 接字串

st = st.replace("\n"," ") #取代\n
st

In [ ]:
import re
def group_words(s):
    return re.findall('[\u4e00-\u9fff]|[a-zA-Z0-9]+',s) #中文英文分開
words = group_words(st)
print(words[:50])

In [ ]:
# ngram 出現次數
def getNgramCount_zip(words,n):
    ngram = list(zip(*[words[i:] for i in range(n)]))
    output = dict()
    for i in range(len(ngram)):
        newNgram = "".join(ngram[i])
        if newNgram in output:
            output[newNgram] += 1
        else:
            output[newNgram] = 1
    return output
two_grams = getNgramCount_zip(words,2)

# from collections import OrderedDict
# order = OrderedDict(sorted(two_grams.items(),key=lambda t: t[1],reverse =True))
# # printdict(order,10)

In [ ]:
get_ipython().magic(u'matplotlib inline')
from matplotlib.font_manager import FontProperties
myfont = FontProperties(fname=r'NotoSerifCJKtc-Black.otf')

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font=myfont.get_family())
sns.set_style("whitegrid",{"font.sans-serif":['Microsoft JhengHei']})

In [ ]:
# 用ngram出現次數 產生文字雲 
from wordcloud import WordCloud
plt.figure()
wordcloud = WordCloud(font_path='./data/NotoSerifCJKtc-Black.otf',max_words=50).generate_from_frequencies(two_grams)

plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# !pip install jieba

#斷詞
import jieba
jieba.load_userdict('dict.txt.big')
segment = list(jieba.cut(st))
print(segment[:10])

In [ ]:
#接回字串
seg_text = ' '.join(segment)
print(seg_text[:100])